<a href="https://colab.research.google.com/github/tpgus2603/DataMining/blob/main/RecommendSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
cd ./drive/MyDrive/json_csv_files/amazon_review2

[Errno 2] No such file or directory: './drive/MyDrive/json_csv_files/amazon_review2'
/content/drive/MyDrive/json_csv_files/amazon_review2


통합데이터에서 전처리하여 필요한 데이터셋 확보


In [ ]:
import json
from collections import Counter
import random

# 경로 설정
products_file_path = './combined_data.json'

# 파일 읽기
with open(products_file_path, 'r', encoding='utf-8') as file:
    data = [json.loads(line) for line in file]

# 리뷰를 가장 많이 남긴 유저 찾기
user_reviews = Counter([review['user_id'] for review in data])
most_reviewed_user = user_reviews.most_common(1)[0][0]

# 가장 많은 리뷰를 남긴 유저의 리뷰 추출
top_user_reviews = [review for review in data if review['user_id'] == most_reviewed_user]

# rating이 5인 리뷰 필터링
five_star_reviews = [review for review in top_user_reviews if review['rating'] == 5]

# 카테고리별로 5개씩 선택
categories = ['Movies_TV', 'Grocery_Gourmet_Food', 'Electronics']
user_reviews = []
selected_reviews = set()
for category in categories:
    category_reviews = [review for review in five_star_reviews if review['category'] == category]
    selected = random.sample(category_reviews, min(5, len(category_reviews)))
    user_reviews.extend(selected)
    selected_reviews.update(map(json.dumps, selected))  # Use JSON string for hashing

# 나머지 리뷰를 test_reviews로 분류
test_reviews = [review for review in five_star_reviews if json.dumps(review) not in selected_reviews]

# 결과 출력
print(f"Most reviewed user ID: {most_reviewed_user}")
print(f"Total reviews by user: {len(top_user_reviews)}")
print(f"Five-star reviews by user: {len(five_star_reviews)}")
print(f"Selected user reviews: {len(user_reviews)}")
print(f"Remaining test reviews: {len(test_reviews)}")

# 선택된 user_reviews 저장 (line JSON 형태로 저장)
user_reviews_path = './user_reviews.json'
with open(user_reviews_path, 'w', encoding='utf-8') as file:
    for review in user_reviews:
        file.write(json.dumps(review, ensure_ascii=False) + '\n')

# 나머지 test_reviews 저장 (line JSON 형태로 저장)
test_reviews_path = './test_reviews.json'
with open(test_reviews_path, 'w', encoding='utf-8') as file:
    for review in test_reviews:
        file.write(json.dumps(review, ensure_ascii=False) + '\n')

print(f"User reviews saved to {user_reviews_path}")
print(f"Test reviews saved to {test_reviews_path}")

Most reviewed user ID: AFZUK3MTBIBEDQOPAK3OATUOUKLA
Total reviews by user: 1531
Five-star reviews by user: 1064
Selected user reviews: 15
Remaining test reviews: 1049
User reviews saved to ./user_reviews.json
Test reviews saved to ./test_reviews.json


In [ ]:
# 라이브러리 설치 (필요시)
!pip install pandas scikit-learn

# 라이브러리 임포트
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

시나리오
1.   트위터에서 추출한 사용자의 프로필이 Movies_TV, Grocery_Gourmet_Food,Eletronics 라고 가정
2.   해당 사용자에게 content based와 colaborative filtering을 이용하여 각각 추천

# Content based

In [ ]:
# 상품 데이터 파일 경로
products_file_path = './combined_data.json'  # 실제 경로로 변경
products_df = pd.read_json(products_file_path, lines=True)

# 원하는 카테고리 목록
desired_categories = ['Movies_TV', 'Grocery_Gourmet_Food', 'Electronics']

# 카테고리 필터링
filtered_products_df = products_df[products_df['category'].isin(desired_categories)].copy()
filtered_products_df = filtered_products_df[['rating', 'product_title', 'category', 'parent_asin']]
print("Filtered Products DataFrame:")
print(filtered_products_df.head())

# user_reviews 데이터 로드
user_reviews_file_path = './user_reviews.json'  # 실제 경로로 변경
user_reviews_df = pd.read_json(user_reviews_file_path, lines=True)
user_reviews_df = user_reviews_df[['product_title', 'category', 'parent_asin']]
user_reviews_df = user_reviews_df[user_reviews_df['category'].isin(desired_categories)].copy()
print("User Reviews DataFrame:")
print(user_reviews_df.head())

# test_reviews 데이터 로드
test_reviews_file_path = './test_reviews.json'  # 실제 경로로 변경
test_reviews_df = pd.read_json(test_reviews_file_path, lines=True)
test_reviews_df = test_reviews_df[['product_title', 'category', 'parent_asin']]
test_reviews_df = test_reviews_df[test_reviews_df['category'].isin(desired_categories)].copy()
print("Test Reviews DataFrame:")
print(test_reviews_df.head())
def combine_product_title(df):
    combined = df['product_title'].fillna('')
    # 빈 문자열을 'No Title'로 대체
    combined = combined.apply(lambda x: x if x.strip() else 'No Title')
    return combined

# test_reviews를 Train/Val로 분할 (7:3)
train_reviews, val_reviews = train_test_split(test_reviews_df, test_size=0.3, random_state=42)


#아이템 셋에서 평가를 위한 테스트셋의 훈련셋 제외
excluded_asins = set(train_reviews['parent_asin'])
filtered_products_df = filtered_products_df[~filtered_products_df['parent_asin'].isin(excluded_asins)]

print(f"Excluded {len(excluded_asins)} ASINs from the combined data.")



# 평가 함수들
def precision_at_k(recommended_asins, val_asins, k=5):
    recommended_top_k = recommended_asins[:k]
    relevant = [asin for asin in recommended_top_k if asin in val_asins]
    precision = len(relevant) / k
    return precision

def recall_at_k(recommended_asins, val_asins, k=5):
    recommended_top_k = recommended_asins[:k]
    relevant = [asin for asin in recommended_top_k if asin in val_asins]
    recall = len(relevant) / len(val_asins) if len(val_asins) > 0 else 0
    return recall

Filtered Products DataFrame:
       rating                                      product_title     category  \
40000       3  Binoculars, 12x42 Binoculars for Adults, Binoc...  Electronics   
40001       1  Toys for 4-5 Year Old Boys, Mom&myaboys 8 X 21...  Electronics   
40002       5  Senso Bluetooth Headphones, Best Wireless Spor...  Electronics   
40003       5  Targus Air Traveler Laptop Backpack, Professio...  Electronics   
40004       5  Edifier P841 Comfortable Noise Isolating Over-...  Electronics   

      parent_asin  
40000  B083NRGZMM  
40001  B07N69T6TM  
40002  B01G8JO5F2  
40003  B001OC5JKY  
40004  B07CJYMRWM  
User Reviews DataFrame:
                                      product_title   category parent_asin
0            Mulan (Disney Gold Classic Collection)  Movies_TV  B00001QEBQ
1                                              None  Movies_TV  B01MU84AWP
2  It's a Wonderful Life (60th Anniversary Edition)  Movies_TV  B000HEWEJO
3                                       

In [ ]:
# 추천 개수 및 평점 임계값 설정
top_n = 3
def get_recommendations_pure_similarity(user_tfidf, tfidf_matrix, category_df, user_reviews_df, asin_to_index, top_n=5):
    """
    순수 유사도 계산.
    """
    cosine_sim_user = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    user_asins = user_reviews_df['parent_asin'].tolist()
    user_indices = [asin_to_index[asin] for asin in user_asins if asin in asin_to_index]

    # 유사도 정렬
    sorted_indices = cosine_sim_user.argsort()[::-1]
    sorted_indices = [idx for idx in sorted_indices if idx not in user_indices]
    top_indices = sorted_indices[:top_n]

    # 추천 상품 반환
    recommended_products = category_df.iloc[top_indices]
    return recommended_products.head(top_n)


In [ ]:

# 추천 로직 수행
for target_category in desired_categories:
    print("\n" + "="*50)
    print(f"Category: {target_category}")
    print("="*50)

    # 카테고리 데이터 준비
    category_df = filtered_products_df[filtered_products_df['category'] == target_category].copy()
    category_df['combined_text'] = combine_product_title(category_df)

    # Train/Val 리뷰 데이터 필터링
    category_train_reviews = train_reviews[train_reviews['category'] == target_category].copy()
    category_val_reviews = val_reviews[val_reviews['category'] == target_category].copy()

    category_train_reviews['combined_text'] = combine_product_title(category_train_reviews)
    category_val_reviews['combined_text'] = combine_product_title(category_val_reviews)

    # TF-IDF 벡터화
    # TF-IDF 벡터화 개선
    tfidf = TfidfVectorizer(stop_words='english', max_features=20000, ngram_range=(1, 2))
    tfidf_matrix = tfidf.fit_transform(category_df['combined_text'])

    # 인덱스 재설정 및 asin_to_index 생성
    category_df = category_df.reset_index(drop=True)
    asin_to_index = pd.Series(category_df.index, index=category_df['parent_asin']).to_dict()

    print("TF-IDF Matrix Shape:", tfidf_matrix.shape)

    # 사용자 리뷰가 있는 경우 쿼리 생성
    if len(category_train_reviews) > 0:
        user_query = ' '.join(category_train_reviews['combined_text'].tolist())
        user_tfidf = tfidf.transform([user_query])
    else:
        user_query = ''
        user_tfidf = tfidf.transform([''])

    val_asins = category_val_reviews['parent_asin'].unique().tolist()

    # 순수 유사도 기반 추천
    recommended_products_similarity = get_recommendations_pure_similarity(
        user_tfidf, tfidf_matrix, category_df, category_train_reviews, asin_to_index, top_n=top_n
    )
    recommended_asins_similarity = recommended_products_similarity['parent_asin'].tolist()
    print(recommended_asins_similarity)
    precision_sim = precision_at_k(recommended_asins_similarity, val_asins, k=top_n)
    recall_sim = recall_at_k(recommended_asins_similarity, val_asins, k=top_n)

    print("순수 유사도 기반 추천 성능:")
    print(f"Precision@{top_n}: {precision_sim}, Recall@{top_n}: {recall_sim}")
        # rating 기반 추천


Category: Movies_TV
TF-IDF Matrix Shape: (9783, 13997)
['B0B5NVVJYH', 'B07CSFX9TG', 'B001CNPXUG']
순수 유사도 기반 추천 성능:
Precision@3: 0.0, Recall@3: 0.0

Category: Grocery_Gourmet_Food
TF-IDF Matrix Shape: (8417, 20000)
['B0BHJNBWSG', 'B006OCW7DQ', 'B0B5GB55GW']
순수 유사도 기반 추천 성능:
Precision@3: 0.0, Recall@3: 0.0

Category: Electronics
TF-IDF Matrix Shape: (9827, 20000)
['B00005JLS3', 'B08FXL1HBL', 'B09RZFJT88']
순수 유사도 기반 추천 성능:
Precision@3: 0.0, Recall@3: 0.0


In [ ]:
# 사용자 리뷰 기반 추천 (성능 평가 없음)
print("\n" + "="*50)
print("User Review 기반 추천 (성능평가 없음)")
print("="*50)

# 특정 사용자 데이터
user_reviews_df['combined_text'] = combine_product_title(user_reviews_df)

for target_category in desired_categories:
    print("\nCategory:", target_category)
    category_df = filtered_products_df[filtered_products_df['category'] == target_category].copy()
    category_df['combined_text'] = combine_product_title(category_df)

    # TF-IDF 벡터화 (max_features 줄임)
    tfidf = TfidfVectorizer(stop_words=None, max_features=5000)  # 스탑워드 제거하지 않음
    tfidf_matrix = tfidf.fit_transform(category_df['combined_text'])

    # 인덱스 재설정 및 asin_to_index 생성
    category_df = category_df.reset_index(drop=True)
    asin_to_index = pd.Series(category_df.index, index=category_df['parent_asin']).to_dict()

    # 평점 기준 필터링
    high_rating_products_df = category_df[category_df['rating'] >= rating_threshold].copy()

    # 사용자 전체 리뷰 텍스트로 TF-IDF 벡터 생성
    user_query_all = ' '.join(user_reviews_df['combined_text'].tolist())
    user_tfidf_all = tfidf.transform([user_query_all])
    # 순수 유사도 기반 추천 (user_review)
    recommended_products_similarity_user = get_recommendations_pure_similarity(
        user_tfidf_all, tfidf_matrix, category_df, user_reviews_df, asin_to_index, top_n=top_n
    )
    print("User Review 기반 순수 유사도 추천 상품:")
    if not recommended_products_similarity_user.empty:
        print(recommended_products_similarity_user[['parent_asin', 'product_title', 'category']])
    else:
        print("추천 없음")


User Review 기반 추천 (성능평가 없음)

Category: Movies_TV
User Review 기반 순수 유사도 추천 상품:
     parent_asin product_title   category
2135  B09GR3G9JY      Free Guy  Movies_TV
4934  B09GR3G9JY      Free Guy  Movies_TV
192   B00HVBB140  Wizard of Oz  Movies_TV

Category: Grocery_Gourmet_Food
User Review 기반 순수 유사도 추천 상품:
     parent_asin                                      product_title  \
6637  B0168CTPZY  PUR 100% Xylitol Chewing Gum, Spearmint, 55 Pi...   
4479  B09LB927T3  The Humble Co. Natural Chewing Gum, 12pk – Ket...   
2528  B09LB927T3  The Humble Co. Natural Chewing Gum, 12pk – Ket...   

                  category  
6637  Grocery_Gourmet_Food  
4479  Grocery_Gourmet_Food  
2528  Grocery_Gourmet_Food  

Category: Electronics
User Review 기반 순수 유사도 추천 상품:
     parent_asin                                      product_title  \
500   B07C3PQHC7  Renegade Concepts: Lap PRO - Stand/Caddy Unive...   
1521  B017O1BWMW  InstaNatual Natural Deodorant for Underarms - ...   
545   B01NAWBY12  Sony Cfd

# Colaborative Filtering

In [ ]:
# local에 padnas, numpy, matplotlib, surprise, sklearn 설치필요
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split

import seaborn as sns

데이터 전처리

In [77]:
def preprocess_review(review_path, min_user_cnt, min_review_cnt):
    review_df = pd.read_json(review_path, lines=True)
    filtered_review_df = review_df[["parent_asin", "rating", "user_id", "title", "category", "product_title"]]
    # filtered_review_df = filtered_review_df[filtered_review_df["category"] == category]
    distinct_review_df = filtered_review_df.drop_duplicates(subset=["user_id", "parent_asin"])

    review_counts = distinct_review_df["parent_asin"].value_counts()
    user_counts = distinct_review_df['user_id'].value_counts()

    result_df = distinct_review_df[(
        (distinct_review_df['user_id'].isin(user_counts[user_counts >= min_user_cnt].index)) &
        (distinct_review_df['parent_asin'].isin(review_counts[review_counts >= min_review_cnt].index))
    )]

    # # 리뷰 개수가 많은 순으로 정렬
    # sorted_parent_asin = result_df["parent_asin"].value_counts().index
    # result_df = result_df.set_index("parent_asin").loc[sorted_parent_asin].reset_index()

    return result_df

In [78]:
def knn_predict(item_sim_df, train_matrix, k=5):
    item_ids = train_matrix.index
    user_ids = train_matrix.columns
    predictions = np.zeros(train_matrix.shape)

    for item_idx, item in enumerate(item_ids):
        for user_idx, user in enumerate(user_ids):
            user_ratings = train_matrix.iloc[:, user_idx].values  # 해당 사용자의 모든 평점
            relevant_items = item_sim_df.iloc[item_idx].values  # 현재 아이템과 다른 아이템 간 유사도

            # K개의 가장 유사한 이웃 선택
            neighbors_idx = np.argsort(relevant_items)[-k:]
            neighbors_sim = relevant_items[neighbors_idx]
            neighbors_ratings = user_ratings[neighbors_idx]

            # 가중합 계산
            weighted_sum = np.dot(neighbors_sim, neighbors_ratings)
            sim_sum = np.abs(neighbors_sim).sum()

            if sim_sum > 0:
                predictions[item_idx, user_idx] = weighted_sum / sim_sum
            else:
                # 유사도가 없는 경우 행 평균값으로 대체
                row_mean = train_matrix.iloc[item_idx, :].mean()
                predictions[item_idx, user_idx] = row_mean

    return predictions

def knn_based_cf(df, org_df, k=5):
    # 사용자-아이템 행렬 생성
    item_user_matrix = df.pivot(index='parent_asin', columns='user_id', values='rating').fillna(0)
    item_ids = item_user_matrix.index
    user_ids = item_user_matrix.columns

    if df.empty:
        return "Empty","Empty"

    # 훈련-테스트 데이터 분할
    trainset, testset = train_test_split(df, test_size=0.2, random_state=42)

    # 훈련 데이터로 사용자-아이템 행렬 생성
    train_matrix = trainset.pivot(index='parent_asin', columns='user_id', values='rating').reindex(index=item_ids, columns=user_ids).fillna(0)
    test_matrix = testset.pivot(index='parent_asin', columns='user_id', values='rating').reindex(index=item_ids, columns=user_ids).fillna(0)

    # 아이템 기반 코사인 유사도 계산
    item_sim_matrix = cosine_similarity(train_matrix)
    np.fill_diagonal(item_sim_matrix, 0)  # 자신과의 유사도는 0으로 설정
    item_sim_df = pd.DataFrame(item_sim_matrix, index=item_ids, columns=item_ids)

    # 예측값 생성
    predictions = knn_predict(item_sim_df, train_matrix, k)

    # 테스트 세트에서 RMSE 계산
    test_actual = test_matrix.values[test_matrix > 0]
    test_predicted = predictions[test_matrix > 0]
    rmse = sqrt(mean_squared_error(test_actual, test_predicted))

    # 커버리지 계산
    total_len = len(org_df['parent_asin'].unique())
    non_zero_predictions = np.sum(predictions > 0)
    coverage_value = non_zero_predictions / total_len
    return rmse, coverage_value

데이터를 전처리 할 때 파라미터로 상품별 최소 리뷰 개수와 사용자별 최소 작성 리뷰 개수를 전달해 데이터를 필터링하였다. 이는 2가지 이유가 있는데, 첫번째는 데이터의 희소성을 줄여 성능 향상을 도모하기 위해서이고, 두번째는 연산 속도를 높이기 위함이었다. 설정한 값은 각각 8로 하였다.

In [79]:
review_path = './combined_data.json'
review_df = pd.read_json(review_path, lines=True)
raw_df = review_df[["parent_asin", "rating", "user_id", "title", "category", "product_title"]]
raw_df = raw_df.drop_duplicates(subset=["user_id", "parent_asin"])
review_df = preprocess_review(review_path,8,8)

아이템-사용자 행렬에서 아이템 끼리의 코사인 유사도를 구한 후 K개의 근접한 이웃을 뽑아 평점 예측을 하였다. 성능 지표로는 정확성을 위한 테스트 셋과 예측으로 만들어진 행렬간의 RMSE와 다양성을 위한 전체 상품 수 중 몇 가지를 추천하는지 그 비율을 나타내는 Coverage 2가지를 채택하였다.

하지만 처음 시도한 결과는 좋지 않았다. RMSE가 4.281과 Coverage가 48.61%라는 만족스럽지 못한 수치가 나왔다. 이는 사용한 샘플 데이터의 희소성이 높기 때문이라고 가정하고 이를 낮추는 것을 목표로 삼았다.

In [80]:
# 아이템 기반 CF 실행
rmse,coverage_value = knn_based_cf(review_df,raw_df)
print("KNN based")
print(f"RMSE: {rmse:.3f}")
print(f"Coverage: {coverage_value:.2%}")

KNN based
RMSE: 4.281
Coverage: 48.61%


샘플 데이터의 희소성을 낮추기 위하여 작성한 리뷰가 많은 사용자 N명 또는 리뷰가 많았던 상품 N개를 골라 데이터를 조정하여 테스트해보았다. N을 20~50 범위에서 Gridsearch를 이용하여 가장 좋은 성능일 때의 파라미터를 찾았다. 그 결과, 인기 사용자는 22명, 인기 상품은 23개 일때 각각 RMSE가 3.808, 3.826이 나왔다. 이렇게 샘플 데이터를 필터링하여 희소성을 낮추자 RMSE 성능이 개선되었다. 하지만 N의 값이 커질수록 coverage는 감소하며 상품 추천의 다양성은 줄어들었다.

In [ ]:
def find_best_hyperparameter_x(review_df, raw_df, y=50, x_range=(20, 50)):
    best_rmse = float('inf')
    best_params = {'x': None, 'rmse': None, 'coverage': None}
    x_values = range(x_range[0], x_range[1] + 1)

    for x in x_values:
        # 상위 x명의 유저와 각 유저당 y개의 리뷰를 필터링
        top_user_id = (
            review_df['user_id']
            .value_counts()
            .head(x)
            .index
        )
        filtered_df = (
            review_df[review_df['user_id'].isin(top_user_id)]
            .groupby('user_id')
            .apply(lambda group: group.head(y))
            .reset_index(drop=True)
        )

        rmse, coverage_value = knn_based_cf(filtered_df, raw_df)
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = {
                'x': x,
                'rmse': rmse,
                'coverage': coverage_value
            }

    return best_params

# 사용 예제
best_params = find_best_hyperparameter_x(review_df, raw_df, y=50, x_range=(20, 50))

print("\nBest Hyperparameters:")
print(f"x: {best_params['x']}")
print(f"RMSE: {best_params['rmse']:.3f}")
print(f"Coverage: {best_params['coverage']:.2%}")

<ipython-input-36-55f2607c5809>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.head(y))
<ipython-input-36-55f2607c5809>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.head(y))
<ipython-input-36-55f2607c5809>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of panda


Best Hyperparameters:
x: 22
RMSE: 3.808
Coverage: 0.97%


In [ ]:
def find_best_hyperparameter_x(review_df, raw_df, y=50, x_range=(20, 50)):
    best_rmse = float('inf')
    best_params = {'x': None, 'rmse': None, 'coverage': None}

    x_values = range(x_range[0], x_range[1] + 1)

    for x in x_values:
        # 상위 N개의 상품과 각 상품의 y개를 필터링
        top_item_id = (
            review_df['parent_asin']
            .value_counts()
            .head(x)
            .index
        )
        filtered_df = (
            review_df[review_df['parent_asin'].isin(top_item_id)]
            .groupby('parent_asin')
            .apply(lambda group: group.head(y))
            .reset_index(drop=True)
        )

        rmse, coverage_value = knn_based_cf(filtered_df, raw_df)
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = {
                'x': x,
                'rmse': rmse,
                'coverage': coverage_value
            }

    return best_params

# 사용 예제
best_params = find_best_hyperparameter_x(review_df, raw_df, y=50, x_range=(20, 50))

print("\nBest Hyperparameters:")
print(f"x: {best_params['x']}")
print(f"RMSE: {best_params['rmse']:.3f}")
print(f"Coverage: {best_params['coverage']:.2%}")


<ipython-input-37-c7e0341740e8>:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.head(y))
<ipython-input-37-c7e0341740e8>:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.head(y))
<ipython-input-37-c7e0341740e8>:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of panda


Best Hyperparameters:
x: 23
RMSE: 3.826
Coverage: 3.22%


**surprise 라이브러리 이용**

GridSearch로도 RMSE에서 좋은 성능을 얻지 못하자 자체 구현한 알고리즘의 최적화 부분에서 문제가 있을 거라는 가설을 세웠다. 그래서 surprise 라이브러리를 사용하여 다시 진행해보았다.

In [82]:
!pip install surprise
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

def prepare_data(df):
    reader = Reader(rating_scale=(1, 5))  # 평점 범위 지정
    data = Dataset.load_from_df(df[["user_id", "parent_asin", "rating"]], reader)
    return data

def coverage(predictions, total_len):
    recommended_items = set([pred.iid for pred in predictions])
    return len(recommended_items) / total_len

def knn_based_cf_surprise(df,org_df):
    data = prepare_data(df)
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

    # 코사인 유사도를 사용한 아이템 기반 CF 모델 설정
    sim_options = {
        "name": "cosine",
        "user_based": False,
    }
    algo = KNNBasic(sim_options=sim_options)
    algo.fit(trainset)

    predictions = algo.test(testset)
    # RMSE 계산
    rmse = accuracy.rmse(predictions)

    # Coverage 계산
    total_len = len(org_df['parent_asin'].unique())
    coverage_value = coverage(predictions, total_len)

    return rmse,coverage_value,trainset

surprise 라이브러리를 사용하니 RMSE가 1.219으로 줄었으나 coverage가 0.36%로 매우 낮아졌다. 이는 처음에 데이터 전처리를 할 때 파라미터(상품별 최소 리뷰 개수, 사용자별 최소 작성 리뷰 개수)를 각각 8개로 설정해 필터링했는데 surprise 라이브러리를 사용하니 다양성이 크게 훼손되었다. 하지만 RMSE 성능이 개선되었고 연산속도가 빨라져 더 큰 데이터를 처리할 수 있다고 판단하여 데이터 전처리 과정에서 파라미터를 2,2로 설정하여 성능을 다시 측정하였다. 그 결과 RMSE가 1.242, Coverage가 5.00%라는 좋은 성능을 얻을 수 있었다.

In [ ]:
rmse,coverage_value,trainset = knn_based_cf_surprise(review_df,raw_df)
print("KNN based")
print(f"RMSE: {rmse:.3f}")
print(f"Coverage: {coverage_value:.2%}")

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.2190
KNN based
RMSE: 1.219
Coverage: 0.36%


In [ ]:
new_review_df = preprocess_review(review_path,2,2)
rmse,coverage_value,trainset = knn_based_cf_surprise(new_review_df,raw_df)
print("KNN based")
print(f"RMSE: {rmse:.3f}")
print(f"Coverage: {coverage_value:.2%}")

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.2421
KNN based
RMSE: 1.242
Coverage: 5.00%


여기서 추가적인 성능 개선을 위하여 KNN에 기반한 Memory based 방식의 CF가 아닌, SVD에 기반한 Model based 방식의 CF를 적용해보았다. 그 결과 RMSE가 1.143으로 개선된 것을 확인할 수 있었다. 이는 데이터의 희소성을 해소했기 때문이다.

In [ ]:
from surprise import SVD

def svd_based_cf(df,org_df):
    data = prepare_data(df)
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

    algo = SVD(n_factors=10, random_state=42)
    algo.fit(trainset)
    predictions = algo.test(testset)

    # RMSE 계산
    rmse = accuracy.rmse(predictions)

    # Coverage 계산
    total_len = len(org_df['parent_asin'].unique())
    coverage_value = coverage(predictions, total_len)
    return rmse, coverage_value

rmse,coverage_value = svd_based_cf(new_review_df,raw_df)
print("SVD based")
print(f"RMSE: {rmse:.3f}")
print(f"Coverage: {coverage_value:.2%}")

RMSE: 1.1430
SVD based
RMSE: 1.143
Coverage: 5.00%


실제 시나리오

1. 카테고리 입력
2. 해당 카테고리 인기 상품 5개 추출
3. 5개 상품에 대하여 각각 3개씩 추천하여 유사도 높은 순으로 정렬

In [83]:
# 입력한 카테고리에서 인기 상품을 추천하는 함수
def CategoryBasedRecommend(review_df, targetCategory, top_n=5):

    filtered_df = review_df[review_df["category"] == targetCategory]

    if filtered_df.empty:
        print(f"카테고리 '{targetCategory}'에 해당하는 데이터가 없습니다.")
        return pd.DataFrame()

    popular_items = (
        filtered_df.groupby(["parent_asin", "category","product_title"])
        .agg(avg_rating=("rating", "mean"), num_reviews=("user_id", "count"))
        .reset_index()
        .sort_values(by=["num_reviews", "avg_rating"], ascending=[False, False])
    )
    top_items = popular_items.head(top_n)
    return top_items

In [84]:
# 특정 상품과 관련된 추천 3개 생성
def recommend_related_items(df, target_item, top_n=3):
    data = prepare_data(df)
    trainset = data.build_full_trainset()
    algo = SVD(n_factors=50, random_state=42)
    algo.fit(trainset)
    item_factors = algo.qi
    item_ids = trainset._raw2inner_id_items.keys()

    item_factors_df = pd.DataFrame(item_factors, index=item_ids)

    if target_item not in item_factors_df.index:
        raise ValueError(f"Target item {target_item} not found in training data.")
    target_vector = item_factors_df.loc[target_item].values.reshape(1, -1)

    # 코사인 유사도 계산
    similarities = cosine_similarity(target_vector, item_factors_df.values).flatten()

    # 유사도를 데이터프레임으로 변환
    similarity_df = pd.DataFrame({
        "parent_asin": item_factors_df.index,
        "similarity": similarities
    })

    # 대상 아이템 제외 및 상위 N개 추천
    top_related_items = similarity_df[similarity_df["parent_asin"] != target_item] \
        .sort_values(by="similarity", ascending=False) \
        .head(top_n)

    # 추천 결과에 상품 정보 병합
    related_items_df = top_related_items.merge(
        df[["parent_asin", "product_title", "category"]].drop_duplicates(), on="parent_asin", how="left"
    )

    return related_items_df

In [85]:
for input_category in desired_categories:
  top_items = CategoryBasedRecommend(new_review_df,input_category)
  recommend_result = []
  for _, row in top_items.iterrows():
      target_item = row["parent_asin"]
      target_category = row["category"]
      filtered_df = new_review_df[new_review_df["category"] == target_category]
      filtered_df = new_review_df[new_review_df["product_title"].notnull()]
      result = recommend_related_items(filtered_df, target_item)
      for _, recommend_row in result.iterrows():
          recommended_asin = recommend_row["parent_asin"]
          if recommended_asin not in [item["parent_asin"] for item in recommend_result]:
              recommend_result.append({
                  "parent_asin": recommended_asin,
                  "product_title": recommend_row["product_title"],
                  "category": recommend_row["category"],
                  "similarity": recommend_row["similarity"]
              })

  sorted_recommendations = sorted(recommend_result, key=lambda x: x["similarity"], reverse=True)[:3]

  print("Input Category : ",input_category,"\n")

  print("Top 3 Recommended Items:")
  for item in sorted_recommendations:
      print(f"Product Title: {item['product_title']}")
      print(f"Similarity: {item['similarity']:.3f}")
      print("-" * 30)


Input Category :  Movies_TV 

Top 3 Recommended Items:
Product Title: MapQuest: Maps and Directions
Similarity: 0.554
------------------------------
Product Title: EHDIS Vinyl Scraper 4 inch Felt Squeegee for Vinyl,Window Tinting, Sign Marking, Film Installing,Decal Applicator Tool - Pack of 2-White
Similarity: 0.540
------------------------------
Product Title: Invisible Glass 95160 2-Piece Quick Change Reach and Clean Tool a Window and Windshield Wand Glass Cleaning Tool for Those Hard-to-Reach Places in Your Car and Home Washable Reversible and Extendable
Similarity: 0.532
------------------------------
Input Category :  Grocery_Gourmet_Food 

Top 3 Recommended Items:
Product Title: SoundAsleep Dream Series Luxury Air Mattress with ComfortCoil Technology & Built-in High Capacity Pump for Home & Camping- Double Height, Adjustable, Inflatable Blow Up, Portable - Twin Size
Similarity: 0.575
------------------------------
Product Title: High Road SwingAway Car Front Seat Organizer with 